## Text preprocessing

In [8]:
import re
import nltk

import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

from functools import lru_cache
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

In [9]:
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []
    
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=MorphAnalyzer()):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

## Load train-test data

In [13]:
train_data = pd.read_csv("data/lenta-ru-train.csv")[:1000]
test_data = pd.read_csv("data/lenta-ru-test.csv")

## Extract lemmas from clean texts

In [41]:
with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, train_data['text']), total=len(train_data)))
    title_lemmas = list(tqdm(p.imap(clean_text, train_data['title']), total=len(train_data)))
    
train_data['lemmas'] = lemmas
train_data['title_lemmas'] = title_lemmas
train_data.sample(5)

100%|██████████| 1000/1000 [00:00<00:00, 1667.09it/s]


,title,text,topic,topic_label,lemmas,title_lemmas
828,Древних людей обвинили в вымирании гигантских ...,Ларс Верделин (Lars Werdelin) из Шведского муз...,Наука и техника,2,ларс верделина lars werdelin шведский музей ес...,древние человек обвинить вымирание гигантский ...
393,Малкин избежал дисквалификации за драку на льду,"Российский нападающий клуба НХЛ ""Питтсбург Пин...",Спорт,1,российский нападать клуб питтсбург пингвинс ев...,малкин избежать дисквалификация драка
339,В Норвегии сыгран рекордный по продолжительнос...,Норвежские клубы «Сторхамар» и «Спарта» в матч...,Спорт,1,норвежский клуб сторхамар спарта матч плей выс...,норвегия сыгранный рекордный продолжительность...
564,Наставник футбольной сборной Испании оштрафова...,Главный тренер сборной Испании по футболу Луис...,Спорт,1,главный тренер сборный испания футбол луис ара...,наставник футбольный сборный испания оштрафова...
477,Возмущения Солнца превратили в песню,"Астрономы записали ""песню Солнца"" - им удалось...",Наука и техника,2,астроном записать песня солнце удаться преобра...,возмущение солнце превратить песня


In [48]:
with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, test_data['text']), total=len(test_data)))
    
test_data['lemmas'] = lemmas
test_data.sample(5)

100%|██████████| 51660/51660 [10:14<00:00, 84.00it/s]


,title,text,lemmas
19994,"""Челси"" не сумел купить бразильского форварда ...","Бразильский футбольный клуб ""Сантос"" отклонил ...",бразильский футбольный клуб сантос отклонить п...
16877,Новый сенсор моментально обнаруживает в крови ...,Группа американских ученых из Калифорнийского ...,группа американский учёный калифорнийский унив...
51095,Журналисты отправили героев новой Call of Duty...,Действие следующей Call of Duty будет происход...,действие следующий call duty происходить космо...
6611,Россия снизила ввозные пошлины на шубы в два раза,Российское правительство снизило с 15 сентября...,российский правительство снизить сентябрь став...
27386,Gibson выбрал лучшие металлические песни,Производитель гитар Gibson составил список из ...,производитель гитара gibson составить список х...


In [16]:
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data.lemmas, train_data.topic_label)

In [73]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams

In [74]:
x_train, x_test, y_train, y_test = train_test_split(train_data[["lemmas", "title_lemmas"]], train_data.topic_label)

In [111]:
# vec = CountVectorizer(ngram_range=(1, 1))
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train["lemmas"]) 

In [112]:
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(bow, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [113]:
pred = clf.predict(vec.transform(x_test["lemmas"]))

In [114]:
accuracy_score(y_test, pred)

0.916

In [ ]:
import fasttext

model = fasttext.train_supervised('data.train.txt')

In [60]:
vec = CountVectorizer(ngram_range=(3, 3))
bow = vec.fit_transform(train_data.lemmas) 

In [ ]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, train_data.topic_label)

In [49]:
pred = clf.predict(vec.transform(test_data.lemmas))

In [53]:
res_df = pd.DataFrame()
res_df["id"] = test_data.index
res_df['category'] = pred
res_df[['id','category']].to_csv('dummy_submission.csv', index=False)